In [ ]:
!gdown --id 1egy4mye5Y_XWDRvQCZ1INYh347hrrdx5
!gdown --id 1zNUnOV_uJPhGLdeqa60GI3MXDe0mjoig
!gdown --id 1t8slD-WEiXlaA3MmTPDFiWvXvRRoIA1Z
!gdown --id 19V6pgcpM9JWNHyhzP0QHupNCRW3s6XH3

Downloading...
From: https://drive.google.com/uc?id=1egy4mye5Y_XWDRvQCZ1INYh347hrrdx5
To: /content/train.csv
14.7MB [00:00, 31.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zNUnOV_uJPhGLdeqa60GI3MXDe0mjoig
To: /content/train_label.csv
100% 9.62k/9.62k [00:00<00:00, 7.52MB/s]
Downloading...
From: https://drive.google.com/uc?id=1t8slD-WEiXlaA3MmTPDFiWvXvRRoIA1Z
To: /content/test.csv
3.30MB [00:00, 105MB/s]
Downloading...
From: https://drive.google.com/uc?id=19V6pgcpM9JWNHyhzP0QHupNCRW3s6XH3
To: /content/test_nolabel.csv
100% 1.70k/1.70k [00:00<00:00, 2.92MB/s]


In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
train_label = pd.read_csv('train_label.csv')

test = pd.read_csv('test.csv')
test_nolabel = pd.read_csv('test_nolabel.csv')

### Revenue Calculation

In [ ]:
# number of staying days
train['stays'] = train['stays_in_weekend_nights'] + train['stays_in_week_nights']
test['stays'] = test['stays_in_weekend_nights'] + test['stays_in_week_nights']

In [ ]:
train['children'] = train['children'].fillna(0) # 缺漏值補 0
test['children'] = test['children'].fillna(0)

In [ ]:
train['people_num'] = train['adults'] + train['children'] + train['babies']
test['people_num'] = test['adults'] + test['children'] + test['babies']

In [ ]:
# train 拿掉，但 test 仍然會有，怪怪的
# train = train[ train['stays'] != 0 ]
# train = train[ train['people_num'] != 0 ]

In [ ]:
# calculate revenue
train['revenue'] = train['stays'] * train['adr'] * (1 - train['is_canceled'])
test['revenue'] = None

In [ ]:
train['date'] = pd.to_datetime(train[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(str).apply(' '.join, 1))
test['date'] = pd.to_datetime(test[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(str).apply(' '.join, 1))

In [ ]:
train.head()

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,stays,people_num,revenue,date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,-6.305161,0,0,Check-Out,2015-07-01,0,2.0,-0.000000,2015-07-01
1,1,City Hotel,0,257,2015,July,27,1,0,2,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6.0,NaN,0,Transient,75.052227,0,0,Check-Out,2015-07-03,2,1.0,150.104454,2015-07-01
2,2,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,74.546401,0,0,Check-Out,2015-07-03,2,2.0,149.092802,2015-07-01
3,3,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,76.376288,0,0,Check-Out,2015-07-03,2,2.0,152.752575,2015-07-01
4,4,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,49.411647,0,0,Check-Out,2015-07-03,2,2.0,98.823294,2015-07-01


In [ ]:
predict_train_label = train.groupby('date').sum()['revenue']

predict_train_label = predict_train_label.to_frame()
predict_train_label = (predict_train_label // 10000)
predict_train_label_list = predict_train_label['revenue'].to_list()
train_label_list = train_label['label'].to_list()

In [ ]:
predict_train_label_list == train_label_list

True

### Canellation

In [ ]:
train.head()

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,stays,people_num,revenue,date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,-6.305161,0,0,Check-Out,2015-07-01,0,2.0,-0.000000,2015-07-01
1,1,City Hotel,0,257,2015,July,27,1,0,2,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6.0,NaN,0,Transient,75.052227,0,0,Check-Out,2015-07-03,2,1.0,150.104454,2015-07-01
2,2,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,74.546401,0,0,Check-Out,2015-07-03,2,2.0,149.092802,2015-07-01
3,3,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,76.376288,0,0,Check-Out,2015-07-03,2,2.0,152.752575,2015-07-01
4,4,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,49.411647,0,0,Check-Out,2015-07-03,2,2.0,98.823294,2015-07-01


In [ ]:
test['is_canceled'] = None
test['adr'] = None
test['reservation_status'] = None
test['reservation_status_date'] = None

In [ ]:
train_column_name = list(train.columns)
test = test[train_column_name] # 把 test 的 column 按照 train 做排序

In [ ]:
test.head()

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,stays,people_num,revenue,date
0,91531,City Hotel,None,75,2017,April,13,1,2,5,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,None,0,1,None,None,7,2.0,None,2017-04-01
1,91532,City Hotel,None,208,2017,April,13,1,4,10,2,0.0,0,SC,BRA,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,None,0,1,None,None,14,2.0,None,2017-04-01
2,91533,Resort Hotel,None,12,2017,April,13,1,2,5,2,0.0,0,Undefined,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,40.0,NaN,0,Contract,None,0,1,None,None,7,2.0,None,2017-04-01
3,91534,City Hotel,None,76,2017,April,13,1,2,5,3,0.0,0,BB,BEL,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,None,0,0,None,None,7,3.0,None,2017-04-01
4,91535,City Hotel,None,9,2017,April,13,1,2,4,2,2.0,0,BB,FRA,Online TA,TA/TO,0,0,0,F,F,0,No Deposit,9.0,NaN,0,Transient,None,0,0,None,None,6,4.0,None,2017-04-01


In [ ]:
train_test = pd.concat([train, test], axis= 0) # 一起處理才不會遺漏類別變數

In [ ]:
train_test.index = range(len(train_test)) # 重新編號

In [ ]:
train_test.head()

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,stays,people_num,revenue,date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,-6.30516,0,0,Check-Out,2015-07-01,0,2.0,-0,2015-07-01
1,1,City Hotel,0,257,2015,July,27,1,0,2,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6.0,NaN,0,Transient,75.0522,0,0,Check-Out,2015-07-03,2,1.0,150.104,2015-07-01
2,2,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,74.5464,0,0,Check-Out,2015-07-03,2,2.0,149.093,2015-07-01
3,3,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,76.3763,0,0,Check-Out,2015-07-03,2,2.0,152.753,2015-07-01
4,4,City Hotel,0,257,2015,July,27,1,0,2,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,49.4116,0,0,Check-Out,2015-07-03,2,2.0,98.8233,2015-07-01


### One-hot Encoding

In [ ]:
hotel = pd.get_dummies(train_test['hotel'])
train_test = train_test.join(hotel)
train_test = train_test.drop(columns=['hotel'])

In [ ]:
# 在 testing data 中，只有 2017/4/1 後的資料
train_test = train_test.drop(columns=['arrival_date_year'])
train_test = train_test.drop(columns=['arrival_date_month']) 
# train_test = train_test.drop(columns=['arrival_date_week_number'])

# train_test['arrival_date_month'].replace({'January':1, 
#                                           'February':2, 
#                                           'March':3, 
#                                           'April':4, 
#                                           'May':5, 
#                                           'June':6,
#                                           'July':7,
#                                           'August':8,
#                                           'September':9,
#                                           'October':10,
#                                           'November':11,
#                                           'December':12,}, inplace=True)

In [ ]:
meal = pd.get_dummies(train_test['meal'])
meal.columns = ['meal BB',
                'meal FB',
                'meal HB',
                'meal SC',
                'meal Undefined']

train_test = train_test.join(meal)
train_test = train_test.drop(columns=['meal']) 

In [ ]:
country = pd.get_dummies(train_test['country'])
train_test = train_test.join(country)
train_test = train_test.drop(columns=['country']) 

In [ ]:
market_segment = pd.get_dummies(train_test['market_segment'])
market_segment.columns = ['market_segment Aviation', 
                  'market_segment Complementary', 
                  'market_segment Corporate', 
                  'market_segment Direct', 
                  'market_segment Groups',
                  'market_segment Offline TA/TO', 
                  'market_segment Online TA', 
                  'market_segment Undefined']
train_test = train_test.join(market_segment)
train_test = train_test.drop(columns=['market_segment']) 

In [ ]:
distribution_channel = pd.get_dummies(train_test['distribution_channel'])
distribution_channel.columns = ['distribution_channel Corporate', 
                                'distribution_channel Direct', 
                                'distribution_channel GDS', 
                                'distribution_channel TA/TO', 
                                'distribution_channel Undefined']
train_test = train_test.join(distribution_channel)
train_test = train_test.drop(columns=['distribution_channel']) 

In [ ]:
train_test['reserved_room_type'] == train_test['assigned_room_type']

0         True
1         True
2         True
3         True
4         True
          ... 
119385    True
119386    True
119387    True
119388    True
119389    True
Length: 119390, dtype: bool

In [ ]:
train_test['assign_reserved_same'] = train_test['reserved_room_type'] == train_test['assigned_room_type']

In [ ]:
train_test['assign_reserved_same'].replace({True:1, False:0})

0         1
1         1
2         1
3         1
4         1
         ..
119385    1
119386    1
119387    1
119388    1
119389    1
Name: assign_reserved_same, Length: 119390, dtype: int64

In [ ]:
reserved_room_type = pd.get_dummies(train_test['reserved_room_type'])
reserved_room_type.columns = ['reserved_room_type A',
                              'reserved_room_type B',
                              'reserved_room_type C',
                              'reserved_room_type D',
                              'reserved_room_type E',
                              'reserved_room_type F',
                              'reserved_room_type G',
                              'reserved_room_type H',
                              'reserved_room_type L',
                              'reserved_room_type P']
train_test = train_test.join(reserved_room_type)
train_test = train_test.drop(columns=['reserved_room_type'])

In [ ]:
assigned_room_type = pd.get_dummies(train_test['assigned_room_type'])
assigned_room_type.columns = ['assigned_room_type A',
                              'assigned_room_type B',
                              'assigned_room_type C',
                              'assigned_room_type D',
                              'assigned_room_type E',
                              'assigned_room_type F',
                              'assigned_room_type G',
                              'assigned_room_type H',
                              'assigned_room_type I',
                              'assigned_room_type K',
                              'assigned_room_type L',
                              'assigned_room_type P']
train_test = train_test.join(assigned_room_type)
train_test = train_test.drop(columns=['assigned_room_type'])

In [ ]:
deposit_type = pd.get_dummies(train_test['deposit_type'])

train_test = train_test.join(deposit_type)
train_test = train_test.drop(columns=['deposit_type']) # 有 Deposit 的幾乎都不可退，但可退費與否將影響取消率，因此保留 Non Refund 及 Refundable

In [ ]:
# train_test['agent'].loc[train_test['agent'].isna()] = 'None'
# train_test['agent'].loc[train_test['agent'].isin([9, 'None']) == False] = "Other"

agent = pd.get_dummies(train_test['agent'])
# agent.columns = ['agent 9', 
#                  'agent None',
#                  'agent other',]
train_test = train_test.join(agent)
train_test = train_test.drop(columns=['agent'])

In [ ]:
# train_test['company'].loc[train_test['company'].isna()] = 'None'
# train_test['company'].loc[train_test['company'].isin([40, 'None']) == False] = "Other"

# company = pd.get_dummies(train_test['company'])
# # company_columns = list(company.columns)
# # for i, company_column in enumerate(company_columns):
# #   company_columns[i] = 'company' + str(company_column)
# # company.columns = company_columns
# company.columns = ['company 40', 
#                    'company None',
#                    'company other',]
# train_test = train_test.join(company)
train_test = train_test.drop(columns=['company'])

In [ ]:
customer_type = pd.get_dummies(train_test['customer_type'])
customer_type.columns = ['customer_type Contract', 
                         'customer_type Group', 
                         'customer_type Transient', 
                         'customer_type Transient-Party']
train_test = train_test.join(customer_type)
train_test = train_test.drop(columns=['customer_type'])

### Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
train_test_date = train_test['date']
train_test = train_test.drop(columns=['ID', 'revenue', 'reservation_status', 'reservation_status_date', 'date'])

In [ ]:
train_test_x = train_test.drop(columns=['is_canceled', 'adr'])

# 2 個模型有不同的 y
train_test_is_canceled_y = train_test['is_canceled']
train_test_adr_y = train_test['adr']

In [ ]:
train_test_x.head()

,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,stays,people_num,City Hotel,Resort Hotel,meal BB,meal FB,meal HB,meal SC,meal Undefined,ABW,AGO,AIA,ALB,AND,ARE,ARG,ARM,ASM,ATA,ATF,AUS,AUT,AZE,BDI,BEL,...,441.0,444.0,446.0,449.0,450.0,451.0,453.0,454.0,455.0,459.0,461.0,464.0,467.0,468.0,469.0,472.0,474.0,475.0,476.0,479.0,480.0,481.0,483.0,484.0,492.0,493.0,495.0,497.0,502.0,508.0,509.0,510.0,526.0,527.0,531.0,535.0,customer_type Contract,customer_type Group,customer_type Transient,customer_type Transient-Party
0,342,27,1,0,0,2,0.0,0,0,0,0,3,0,0,0,0,2.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,257,27,1,0,2,1,0.0,0,0,0,0,1,0,0,0,2,1.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,257,27,1,0,2,2,0.0,0,0,0,0,0,0,0,0,2,2.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,257,27,1,0,2,2,0.0,0,0,0,0,0,0,0,0,2,2.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,257,27,1,0,2,2,0.0,0,0,0,0,0,0,0,0,2,2.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
train_test_is_canceled_y.head() 

0    0
1    0
2    0
3    0
4    0
Name: is_canceled, dtype: object

In [ ]:
train_test_adr_y.head() 

0   -6.30516
1    75.0522
2    74.5464
3    76.3763
4    49.4116
Name: adr, dtype: object

In [ ]:
scale = StandardScaler() #z-scaler物件
normalized_x_train_test = pd.DataFrame(scale.fit_transform(train_test_x), columns=train_test_x.keys())

In [ ]:
normalized_x_train_test.head()

,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,stays,people_num,City Hotel,Resort Hotel,meal BB,meal FB,meal HB,meal SC,meal Undefined,ABW,AGO,AIA,ALB,AND,ARE,ARG,ARM,ASM,ATA,ATF,AUS,AUT,AZE,BDI,BEL,...,441.0,444.0,446.0,449.0,450.0,451.0,453.0,454.0,455.0,459.0,461.0,464.0,467.0,468.0,469.0,472.0,474.0,475.0,476.0,479.0,480.0,481.0,483.0,484.0,492.0,493.0,495.0,497.0,502.0,508.0,509.0,510.0,526.0,527.0,531.0,535.0,customer_type Contract,customer_type Group,customer_type Transient,customer_type Transient-Party
0,2.227051,-0.012141,-1.685297,-0.92889,-1.310240,0.247897,-0.260659,-0.081579,-0.18156,-0.10318,-0.091555,4.260101,-0.131924,-0.254873,-0.720694,-1.340370,0.043967,-1.407224,1.407224,0.541627,-0.08203,-0.371266,-0.312954,-0.09944,-0.004093,-0.055148,-0.002894,-0.010026,-0.007657,-0.020673,-0.042375,-0.008186,-0.002894,-0.004093,-0.002894,-0.059841,-0.103402,-0.011934,-0.002894,-0.141453,...,-0.007657,-0.002894,-0.002894,-0.004093,-0.002894,-0.002894,-0.002894,-0.004093,-0.012616,-0.011577,-0.004093,-0.028662,-0.018077,-0.020263,-0.004093,-0.002894,-0.011934,-0.008186,-0.004093,-0.016374,-0.002894,-0.008186,-0.002894,-0.009599,-0.015316,-0.017124,-0.021855,-0.002894,-0.01418,-0.007089,-0.009152,-0.004093,-0.009152,-0.017124,-0.023872,-0.005013,-0.188008,-0.069688,0.576441,-0.516258
1,1.431638,-0.012141,-1.685297,-0.92889,-0.262174,-1.478447,-0.260659,-0.081579,-0.18156,-0.10318,-0.091555,1.194040,-0.131924,-0.254873,-0.720694,-0.558334,-1.340324,0.710619,-0.710619,-1.846291,-0.08203,2.693484,-0.312954,-0.09944,-0.004093,-0.055148,-0.002894,-0.010026,-0.007657,-0.020673,-0.042375,-0.008186,-0.002894,-0.004093,-0.002894,-0.059841,-0.103402,-0.011934,-0.002894,-0.141453,...,-0.007657,-0.002894,-0.002894,-0.004093,-0.002894,-0.002894,-0.002894,-0.004093,-0.012616,-0.011577,-0.004093,-0.028662,-0.018077,-0.020263,-0.004093,-0.002894,-0.011934,-0.008186,-0.004093,-0.016374,-0.002894,-0.008186,-0.002894,-0.009599,-0.015316,-0.017124,-0.021855,-0.002894,-0.01418,-0.007089,-0.009152,-0.004093,-0.009152,-0.017124,-0.023872,-0.005013,-0.188008,-0.069688,0.576441,-0.516258
2,1.431638,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260659,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.254873,-0.720694,-0.558334,0.043967,0.710619,-0.710619,-1.846291,-0.08203,2.693484,-0.312954,-0.09944,-0.004093,-0.055148,-0.002894,-0.010026,-0.007657,-0.020673,-0.042375,-0.008186,-0.002894,-0.004093,-0.002894,-0.059841,-0.103402,-0.011934,-0.002894,-0.141453,...,-0.007657,-0.002894,-0.002894,-0.004093,-0.002894,-0.002894,-0.002894,-0.004093,-0.012616,-0.011577,-0.004093,-0.028662,-0.018077,-0.020263,-0.004093,-0.002894,-0.011934,-0.008186,-0.004093,-0.016374,-0.002894,-0.008186,-0.002894,-0.009599,-0.015316,-0.017124,-0.021855,-0.002894,-0.01418,-0.007089,-0.009152,-0.004093,-0.009152,-0.017124,-0.023872,-0.005013,-0.188008,-0.069688,0.576441,-0.516258
3,1.431638,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260659,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.254873,-0.720694,-0.558334,0.043967,0.710619,-0.710619,-1.846291,-0.08203,2.693484,-0.312954,-0.09944,-0.004093,-0.055148,-0.002894,-0.010026,-0.007657,-0.020673,-0.042375,-0.008186,-0.002894,-0.004093,-0.002894,-0.059841,-0.103402,-0.011934,-0.002894,-0.141453,...,-0.007657,-0.002894,-0.002894,-0.004093,-0.002894,-0.002894,-0.002894,-0.004093,-0.012616,-0.011577,-0.004093,-0.028662,-0.018077,-0.020263,-0.004093,-0.002894,-0.011934,-0.008186,-0.004093,-0.016374,-0.002894,-0.008186,-0.002894,-0.009599,-0.015316,-0.017124,-0.021855,-0.002894,-0.01418,-0.007089,-0.009152,-0.004093,-0.009152,-0.017124,-0.023872,-0.005013,-0.188008,-0.069688,0.576441,-0.516258
4,1.431638,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.

# Classifier

### Split Train & Test

In [ ]:
train_x, train_is_canceled_y = normalized_x_train_test[:-27859], train_test_is_canceled_y[:-27859]
test_x, test_is_canceled_y = normalized_x_train_test[-27859:], train_test_is_canceled_y[-27859:]

### Data Augmentation

In [ ]:
# import numpy as np

# mu, sigma = 0, 0.01
# noise = np.random.normal(mu, sigma, [train_x.shape[0], train_x.shape[1]]) 
# print(noise)

In [ ]:
# # 增加一倍的資料量
# noisy_train_x = train_x + noise
# augment_train_x = pd.concat([train_x, noisy_train_x])
# augment_train_y = pd.concat([train_y, train_y])

In [ ]:
# import random

# order_list = []
# for i in range(183062): 
#     order_list.append(i)

In [ ]:
# order_list_t = order_list[:82384] + order_list[91530:-9147]
# order_list_v = order_list[82384:91530] + order_list[-9147:]

# print(len(order_list_t))
# print(len(order_list_v))

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification
# from sklearn import ensemble, preprocessing, metrics
# from sklearn.metrics import f1_score

# augment_train_y = augment_train_y.to_list()

# train_x_cross = augment_train_x.iloc[ order_list_t ]
# train_y_cross = [augment_train_y[i] for i in order_list_t] 

# valid_x_cross = augment_train_x.iloc[ order_list_v ]
# valid_y_cross = [augment_train_y[i] for i in order_list_v]

# # training
# forest = ensemble.RandomForestClassifier(n_estimators = 100)
# forest_fit = forest.fit(train_x_cross, train_y_cross)
# y_train_predicted = forest.predict(train_x_cross) 
# accuracy = metrics.accuracy_score(train_y_cross, y_train_predicted)
# print('training accuracy', accuracy)

# # predicting
# y_valid_predicted = forest.predict(valid_x_cross) 
# accuracy = metrics.accuracy_score(valid_y_cross, y_valid_predicted)
# print('validating accuracy', accuracy)
# print('F1 Score')
# print(f1_score(valid_y_cross, y_valid_predicted, average='macro'))
# print(f1_score(valid_y_cross, y_valid_predicted, average='micro'))
# print(f1_score(valid_y_cross, y_valid_predicted, average='weighted'))
# print('')

training accuracy 0.9973417329716148
validating accuracy 0.8158858579784617
F1 Score
0.773028371243609
0.8158858579784617
0.806288886486697



### Training & Validating 
### Random Forest

In [ ]:
import random

order_list = []
for i in range(len(train_x)):
    order_list.append(i)


# 9147 剛好切到 2017/1/31
order_list_t = order_list[:-9147] 
order_list_v = order_list[-9147:]

# 前面 21996 筆是 2015 年
order_list_t = order_list_t[21996:]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import ensemble, preprocessing, metrics
from sklearn.metrics import f1_score

# train_x_cross = train_x.iloc[ order_list_t ]
# train_y_cross = [train_is_canceled_y[i] for i in order_list_t] 

# valid_x_cross = train_x.iloc[ order_list_v ]
# valid_y_cross = [train_is_canceled_y[i] for i in order_list_v]

# # training
# forest = ensemble.RandomForestClassifier(n_estimators=300, random_state=0)
# forest_fit = forest.fit(train_x_cross, train_y_cross)
# y_train_predicted = forest.predict(train_x_cross) 
# accuracy = metrics.accuracy_score(train_y_cross, y_train_predicted)
# print('training accuracy', accuracy)

# # predicting
# y_valid_is_canceled_predicted = forest.predict(valid_x_cross) 
# accuracy = metrics.accuracy_score(valid_y_cross, y_valid_is_canceled_predicted)
# print('validating accuracy', accuracy)
# print('F1 Score')
# print(f1_score(valid_y_cross, y_valid_is_canceled_predicted, average='macro'))
# print(f1_score(valid_y_cross, y_valid_is_canceled_predicted, average='micro'))
# print(f1_score(valid_y_cross, y_valid_is_canceled_predicted, average='weighted'))

# 250 > 300 > 350 2016:250 
# training accuracy 0.9958435450751805
# validating accuracy 0.8232207281075763
# F1 Score
# 0.784531076339654
# 0.8232207281075763
# 0.8153250845192309

# 350 > 300 > 250 > 400 All:350
# training accuracy 0.9946470188386094
# validating accuracy 0.8201596151743741
# F1 Score
# 0.7796485419127992
# 0.8201596151743741
# 0.8115140852610804

### Use All Data

In [ ]:
order_list_t = order_list

In [ ]:
train_x_cross = train_x.iloc[ order_list_t ]
train_y_cross = [train_is_canceled_y[i] for i in order_list_t] 

test_x_cross = test_x

# training
forest_1 = ensemble.RandomForestClassifier(n_estimators=300, random_state=0)
forest_fit = forest_1.fit(train_x_cross, train_y_cross)
y_train_predicted = forest_1.predict(train_x_cross) 
accuracy = metrics.accuracy_score(train_y_cross, y_train_predicted)
print('training accuracy', accuracy)

# predicting
y_test_is_canceled_predicted = forest_1.predict(test_x_cross) 

training accuracy 0.9943407151675389


In [ ]:
cancel_probability = forest_1.predict_proba(test_x_cross) # [cancel, not cancel]
cancel_probability = cancel_probability[:, 1]
print(y_test_is_canceled_predicted) # hard label
print(cancel_probability) # soft label

[0 1 0 ... 0 0 0]
[0.34666667 0.82       0.37787157 ... 0.17666667 0.02333333 0.11333333]


In [ ]:
!gdown --id 1k1RKpuuv-UW5k3t--t3cqy2j-s8kh5V5
!gdown --id 17lsatK1aSZQlSie61x15DRQX5DTVSNNd

adr_test_pred = pd.read_csv('adr_output_20210103.csv', header=None)
ensemble_adr_test_pred = pd.read_csv('adr_output_20210110.csv', header=None)

Downloading...
From: https://drive.google.com/uc?id=1k1RKpuuv-UW5k3t--t3cqy2j-s8kh5V5
To: /content/adr_output_20210103.csv
100% 696k/696k [00:00<00:00, 91.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=17lsatK1aSZQlSie61x15DRQX5DTVSNNd
To: /content/adr_output_20210110.csv
5.02MB [00:00, 161MB/s]


In [ ]:
ensemble_adr_test_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,101.763145,NaN,98.756882,92.611130,90.678856,95.814171,96.545830,90.815102,98.858635,88.173447,94.337479,100.672249,85.889496,97.234131,96.792107,92.602806
1,43.326958,NaN,50.519096,50.593338,44.205395,42.085812,49.701481,40.581783,44.359417,39.808346,34.530357,58.787785,37.797218,46.357811,49.782597,46.039417
2,86.269730,NaN,58.495701,53.513168,61.422215,60.998245,65.094170,61.445824,64.444633,60.927193,65.372490,67.209106,53.300945,61.827183,57.782360,60.640114
3,112.073639,NaN,115.126747,106.661385,107.175003,107.607918,109.834518,99.802322,114.094109,104.080025,107.058823,118.282921,104.191795,111.615074,115.474785,109.097237
4,204.763046,NaN,192.618546,196.715088,197.331558,222.737152,208.099518,186.642776,201.046860,187.077270,191.435318,201.716019,197.836578,204.573654,185.200455,202.911591


In [ ]:
ensemble_adr_test_pred['final_adr'] = (ensemble_adr_test_pred[2] + ensemble_adr_test_pred[3] + ensemble_adr_test_pred[4] + ensemble_adr_test_pred[5] + ensemble_adr_test_pred[6] + ensemble_adr_test_pred[7] + ensemble_adr_test_pred[8] + ensemble_adr_test_pred[9] + ensemble_adr_test_pred[10] + ensemble_adr_test_pred[11] + ensemble_adr_test_pred[12] + ensemble_adr_test_pred[13] + ensemble_adr_test_pred[14] + ensemble_adr_test_pred[15]) / 14

In [ ]:
ensemble_adr_test_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,final_adr
0,101.763145,NaN,98.756882,92.611130,90.678856,95.814171,96.545830,90.815102,98.858635,88.173447,94.337479,100.672249,85.889496,97.234131,96.792107,92.602806,94.270166
1,43.326958,NaN,50.519096,50.593338,44.205395,42.085812,49.701481,40.581783,44.359417,39.808346,34.530357,58.787785,37.797218,46.357811,49.782597,46.039417,45.367847
2,86.269730,NaN,58.495701,53.513168,61.422215,60.998245,65.094170,61.445824,64.444633,60.927193,65.372490,67.209106,53.300945,61.827183,57.782360,60.640114,60.890953
3,112.073639,NaN,115.126747,106.661385,107.175003,107.607918,109.834518,99.802322,114.094109,104.080025,107.058823,118.282921,104.191795,111.615074,115.474785,109.097237,109.293047
4,204.763046,NaN,192.618546,196.715088,197.331558,222.737152,208.099518,186.642776,201.046860,187.077270,191.435318,201.716019,197.836578,204.573654,185.200455,202.911591,198.281599


In [ ]:
adr_test_pred2 = ensemble_adr_test_pred['final_adr']
adr_test_pred = (adr_test_pred + adr_test_pred2) / 2
print(adr_test_pred)

In [ ]:
test['adr_test_pred'] = adr_test_pred
test['y_test_is_canceled_predicted'] = y_test_is_canceled_predicted
test['cancel_probability'] = cancel_probability

In [ ]:
test['predicted_revenue_1'] = test['stays'] * test['adr_test_pred'] * (1 - test['y_test_is_canceled_predicted'])
test['predicted_revenue_2'] = test['stays'] * test['adr_test_pred'] * (1 - test['cancel_probability'])

In [ ]:
import numpy

answer1 = (test.groupby('date').sum() / 10000)['predicted_revenue_1']
answer2 = (test.groupby('date').sum() / 10000)['predicted_revenue_2']
# numpy.savetxt("answer20210109xgboost.csv", answer, delimiter=",")

In [ ]:
answer1

In [ ]:
answer2

###Use Only 2016 & 2017

In [ ]:
order_list_t = order_list[21996:]

In [ ]:
train_x_cross = train_x.iloc[ order_list_t ]
train_y_cross = [train_is_canceled_y[i] for i in order_list_t] 

test_x_cross = test_x

# training
forest = ensemble.RandomForestClassifier(n_estimators=300, random_state=0)
forest_fit = forest.fit(train_x_cross, train_y_cross)
y_train_predicted = forest.predict(train_x_cross) 
accuracy = metrics.accuracy_score(train_y_cross, y_train_predicted)
print('training accuracy', accuracy)

# predicting
y_test_is_canceled_predicted = forest.predict(test_x_cross) 

In [ ]:
cancel_probability = forest.predict_proba(test_x_cross) # [cancel, not cancel]
cancel_probability = cancel_probability[:, 1]

In [ ]:
test['adr_test_pred'] = adr_test_pred
test['y_test_is_canceled_predicted'] = y_test_is_canceled_predicted
test['cancel_probability'] = cancel_probability

In [ ]:
test['predicted_revenue_3'] = test['stays'] * test['adr_test_pred'] * (1 - test['y_test_is_canceled_predicted'])
test['predicted_revenue_4'] = test['stays'] * test['adr_test_pred'] * (1 - test['cancel_probability'])

In [ ]:
answer3 = (test.groupby('date').sum() / 10000)['predicted_revenue_3']
answer4 = (test.groupby('date').sum() / 10000)['predicted_revenue_4']
# numpy.savetxt("answer20210109xgboost.csv", answer, delimiter=",")

In [ ]:
print(answer3)
print(answer4)

In [ ]:
final_answer = (answer1 + answer2 + answer3 + answer4) // 4
print(final_answer)

In [ ]:
numpy.savetxt("final_answer.csv", final_answer, delimiter=",")

In [ ]:
import numpy
import pickle

numpy.savetxt("cancel_output_20210109.csv", y_test_is_canceled_predicted, delimiter=",")
filename = 'Best_RF.sav'
pickle.dump(forest, open(filename, 'wb'))

# Regressor

### Split Train & Test

In [ ]:
# train_x, train_adr_y = normalized_x_train_test[:-27859], train_test_adr_y[:-27859]
# test_x, test_adr_y = normalized_x_train_test[-27859:], train_test_adr_y[-27859:]

### Training & Validating 
### Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

# train_x_cross = train_x.iloc[ order_list_t ]
# train_y_cross = [train_adr_y[i] for i in order_list_t] 

# valid_x_cross = train_x.iloc[ order_list_v ]
# valid_y_cross = [train_adr_y[i] for i in order_list_v]

# # training
# forest = ensemble.RandomForestRegressor(n_estimators=200, 
#                                         criterion='mse', 
#                                         max_depth=None, # depth of tree 
#                                         min_samples_leaf=5, 
#                                         random_state=0)

# forest_fit = forest.fit(train_x_cross, train_y_cross)
# y_train_predicted = forest.predict(train_x_cross)
# mse = mean_squared_error(train_y_cross, y_train_predicted) 
# print('training mse', mse)

# # predicting
# y_valid_adr_predicted = forest.predict(valid_x_cross) 
# mse = mean_squared_error(valid_y_cross, y_valid_adr_predicted) 
# print('validating mse', mse)  

training mse 737.1139786997373
validating mse 826.5598111843717


In [ ]:
# y_valid_is_canceled_predicted

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# y_valid_adr_predicted

array([ 94.34466776, 140.91786071,  67.83401687, ..., 113.42985184,
        48.17642383,  36.99442925])

In [ ]:
# len(y_valid_adr_predicted)

9147

In [ ]:
# train['date'] = pd.to_datetime(train[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].astype(str).apply(' '.join, 1))

In [ ]:
# valid_date_stay = pd.DataFrame(train.iloc[-9147:][['date', 'stays']])

In [ ]:
# valid_date_stay['adr'] = list(y_valid_adr_predicted)
# valid_date_stay['is_canceled'] = list(y_valid_is_canceled_predicted)
# valid_date_stay['not_canceled_revenue'] = valid_date_stay['adr'] * (1 - valid_date_stay['is_canceled']) * valid_date_stay['stays']

In [ ]:
# y_ultimate_valid = valid_date.groupby('date').sum() // 10000
# y_ultimate_valid.head()

,revenue,is_canceled,not_canceled_revenue,adr
date,,,,
2017-02-01,3.0,0,1.0,3.0
2017-02-02,3.0,0,2.0,3.0
2017-02-03,2.0,0,1.0,2.0
2017-02-04,4.0,0,3.0,4.0
2017-02-05,3.0,0,3.0,3.0


In [ ]:
# y_label = train_label.iloc[-len(y_ultimate_valid):]['label']

In [ ]:
# y_ultimate_valid['1'] = 1

In [ ]:
# y_predict = y_ultimate_valid['not_canceled_revenue'] - y_ultimate_valid['1']

In [ ]:
# from sklearn.metrics import mean_absolute_error

# mae = mean_absolute_error(list(y_ultimate_valid['not_canceled_revenue']), y_label)

In [ ]:
# mae = mean_absolute_error(y_predict, y_label)
# mae

0.4406779661016949

### Training & Validating XG-Boost

In [ ]:
import random

order_list = []
for i in range(len(train_x)):
    order_list.append(i)


# 9147 剛好切到 2017/1/31
order_list_t = order_list[:-9147] 
order_list_v = order_list[-9147:]

# 前面 21996 筆是 2015 年
# order_list_t = order_list_t[21996:]

In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn import ensemble, preprocessing, metrics
from sklearn.metrics import f1_score

train_x_cross = train_x.iloc[ order_list_t ]
train_y_cross = [train_is_canceled_y[i] for i in order_list_t] 

valid_x_cross = train_x.iloc[ order_list_v ]
valid_y_cross = [train_is_canceled_y[i] for i in order_list_v]

# training
clf = XGBClassifier(
        n_estimators=300, #樹的個數
        learning_rate= 0.3, # 如同學習率
        max_depth=15, # 構建樹的深度，越大越容易過擬合 
        subsample=1, # 隨機取樣訓練樣本 訓練例項的子取樣比
        gamma=0, # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        reg_lambda=0.1, # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        max_delta_step=0, #最大增量步長，我們允許每個樹的權重估計
        colsample_bytree=1, # 生成樹時進行的列取樣 

        # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
        # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
        #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        min_child_weight=0.1, 
        seed=1000)

clf.fit(train_x_cross, train_y_cross)
y_train_predicted = clf.predict(train_x_cross) 
accuracy = metrics.accuracy_score(train_y_cross, y_train_predicted)
print('training accuracy', accuracy)

# predicting
y_valid_predicted = clf.predict(valid_x_cross) 
accuracy = metrics.accuracy_score(valid_y_cross, y_valid_predicted)
print('validating accuracy', accuracy)
print('F1 Score')
print(f1_score(valid_y_cross, y_valid_predicted, average='macro'))
print(f1_score(valid_y_cross, y_valid_predicted, average='micro'))
print(f1_score(valid_y_cross, y_valid_predicted, average='weighted'))

training accuracy 0.9946470188386094
validating accuracy 0.8191756860172734
F1 Score
0.7866598487937092
0.8191756860172734
0.8147503998535512


###Use All Data

In [ ]:
order_list_t = order_list

train_x_cross = train_x.iloc[ order_list_t ]
train_y_cross = [train_is_canceled_y[i] for i in order_list_t] 

test_x_cross = test_x

# training
clf = XGBClassifier(
        n_estimators=300, #樹的個數
        learning_rate= 0.3, # 如同學習率
        max_depth=15, # 構建樹的深度，越大越容易過擬合 
        subsample=1, # 隨機取樣訓練樣本 訓練例項的子取樣比
        gamma=0, # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        reg_lambda=0.1, # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        max_delta_step=0, #最大增量步長，我們允許每個樹的權重估計
        colsample_bytree=1, # 生成樹時進行的列取樣 

        # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
        # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
        #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        min_child_weight=0.1, 
        seed=1000)

clf.fit(train_x_cross, train_y_cross)
y_train_predicted = clf.predict(train_x_cross) 
accuracy = metrics.accuracy_score(train_y_cross, y_train_predicted)
print('training accuracy', accuracy)

training accuracy 0.9943407151675389


NameError: ignored

In [ ]:
# predicting
y_test_is_canceled_predicted = clf.predict(test_x_cross) 
cancel_probability = clf.predict_proba(test_x_cross) # [cancel, not cancel]
cancel_probability = cancel_probability[:, 1]
print(y_test_is_canceled_predicted) # hard label
print(cancel_probability) # soft label

[0 1 0 ... 0 0 0]
[3.8490891e-03 9.9419057e-01 8.1443973e-02 ... 2.9451087e-01 4.8741649e-06
 3.0763773e-03]


In [ ]:
test['adr_test_pred'] = adr_test_pred
test['y_test_is_canceled_predicted'] = y_test_is_canceled_predicted
test['cancel_probability'] = cancel_probability

test['predicted_revenue_5'] = test['stays'] * test['adr_test_pred'] * (1 - test['y_test_is_canceled_predicted'])
test['predicted_revenue_6'] = test['stays'] * test['adr_test_pred'] * (1 - test['cancel_probability'])

In [ ]:
answer5 = (test.groupby('date').sum() / 10000)['predicted_revenue_5']
answer6 = (test.groupby('date').sum() / 10000)['predicted_revenue_6']

###Use Only 2016

In [ ]:
order_list_t = order_list[21996:]

train_x_cross = train_x.iloc[ order_list_t ]
train_y_cross = [train_is_canceled_y[i] for i in order_list_t] 

test_x_cross = test_x

# training
clf = XGBClassifier(
        n_estimators=300, #樹的個數
        learning_rate= 0.3, # 如同學習率
        max_depth=15, # 構建樹的深度，越大越容易過擬合 
        subsample=1, # 隨機取樣訓練樣本 訓練例項的子取樣比
        gamma=0, # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        reg_lambda=0.1, # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        max_delta_step=0, #最大增量步長，我們允許每個樹的權重估計
        colsample_bytree=1, # 生成樹時進行的列取樣 

        # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
        # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
        #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        min_child_weight=0.1, 
        seed=1000)

clf.fit(train_x_cross, train_y_cross)
y_train_predicted = clf.predict(train_x_cross) 
accuracy = metrics.accuracy_score(train_y_cross, y_train_predicted)
print('training accuracy', accuracy)

training accuracy 0.9952829510318545


In [ ]:
# predicting
y_test_is_canceled_predicted = clf.predict(test_x_cross) 
cancel_probability = clf.predict_proba(test_x_cross) # [cancel, not cancel]
cancel_probability = cancel_probability[:, 1]
print(y_test_is_canceled_predicted) # hard label
print(cancel_probability) # soft label

[0 1 0 ... 0 0 0]
[4.8760241e-03 9.8190248e-01 3.8716555e-02 ... 1.9839603e-01 4.1901229e-05
 6.1603209e-06]


In [ ]:
test['adr_test_pred'] = adr_test_pred
test['y_test_is_canceled_predicted'] = y_test_is_canceled_predicted
test['cancel_probability'] = cancel_probability

test['predicted_revenue_7'] = test['stays'] * test['adr_test_pred'] * (1 - test['y_test_is_canceled_predicted'])
test['predicted_revenue_8'] = test['stays'] * test['adr_test_pred'] * (1 - test['cancel_probability'])

In [ ]:
answer7 = (test.groupby('date').sum() / 10000)['predicted_revenue_7']
answer8 = (test.groupby('date').sum() / 10000)['predicted_revenue_8']

In [ ]:
final_answer = (answer1 + answer2 + answer3 + answer4 + answer5 + answer6 + answer7 + answer8) // 8
print(final_answer)

date
2017-04-01    3.0
2017-04-02    2.0
2017-04-03    4.0
2017-04-04    1.0
2017-04-05    3.0
             ... 
2017-08-27    5.0
2017-08-28    8.0
2017-08-29    3.0
2017-08-30    3.0
2017-08-31    4.0
Length: 153, dtype: float64


In [ ]:
numpy.savetxt("final_answer8.csv", final_answer, delimiter=",")

### Balance or Not

In [ ]:
# 確認資料是否不平衡
print('No_Cancellation Num: ', list(train_y).count(0))
print('Cancellation Num: ', list(train_y).count(1))

No_Cancellation Num:  58771
Cancellation Num:  32760
